In [1]:
%pip install datasets
%pip install evaluate
%pip install datasets
%pip install transformers
%pip install sklearn
%pip install torch

Note: you may need to restart the kernel to use updated packages.


In [1]:
from datasets import Dataset, load_dataset
from utils import Utils
import config
from tokenizer import Tokenizer
import pandas as pd
from transformers import DataCollatorWithPadding
from model import Intentmodel
from dataloader import Dataloader
utils_obj = Utils()
data_obj = Dataloader()
model_obj = Intentmodel()
tokenizer_obj = Tokenizer()
intent_index_mapping = config.INTENT_INDEX_MAPPING

In [2]:
tokenizer = tokenizer_obj.initialize_tokenizer()

In [3]:
def preprocess_function(examples):
    tokenized_batch = tokenizer(examples['text'], truncation=True)
    tokenized_batch["label"] = [intent_index_mapping[label] for label in examples["label"]]
    return tokenized_batch

In [4]:
df = utils_obj.load_data("Dataset/train_data.csv")
df.drop(["path", "object", "location"], axis=1, inplace=True)
train_dataset, test_dataset = data_obj.prepare_dataset(df)
tokenized_train_data = train_dataset.map(preprocess_function, batched=True)
tokenized_test_data = test_dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [2]:
model = model_obj.initialize_model()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifi

In [106]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=config.MODEL_OUTPUT_PATH,
    learning_rate=config.LEARNING_RATE,
    per_device_train_batch_size=config.TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=config.VAL_BATCH_SIZE,
    num_train_epochs=config.EPOCHS,
    weight_decay=config.WEIGHT_DECAY,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_test_data,
    tokenizer=tokenizer,
    data_collator=data_collator
)
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/home/sanket/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 9252
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total 

Step,Training Loss
500,0.185300
1000,0.001800
1500,0.000800
2000,0.000500
2500,0.000300


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1500/special_toke

TrainOutput(global_step=2895, training_loss=0.03263944031249262, metrics={'train_runtime': 2612.2178, 'train_samples_per_second': 17.709, 'train_steps_per_second': 1.108, 'total_flos': 127744940708928.0, 'train_loss': 0.03263944031249262, 'epoch': 5.0})

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 25000
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 7815


TypeError: 'tuple' object is not callable